In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D,Bidirectional
from keras.models import Model
from keras.layers import TimeDistributed
from keras.layers import concatenate
from keras.layers import Dense, Input
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import CuDNNLSTM
import gc
import os

Using TensorFlow backend.


# Data loading

In [2]:
x_train = np.load('/home/tidal/ML_Data/Tweet_Sentiment_Extraction/features/x_train.npy')
x_test = np.load('/home/tidal/ML_Data/Tweet_Sentiment_Extraction/features/x_test.npy')
sent_train = np.load('/home/tidal/ML_Data/Tweet_Sentiment_Extraction/features/sent_train.npy')
sent_test = np.load('/home/tidal/ML_Data/Tweet_Sentiment_Extraction/features/sent_test.npy')
targets = np.load('/home/tidal/ML_Data/Tweet_Sentiment_Extraction/targets/targets.npy')
embedding_matrix_ = np.load('/home/tidal/ML_Data/Tweet_Sentiment_Extraction/transformer_layer/embedding_matrix.npy')

In [3]:
embedding_matrix_

array([[-0.01664949, -0.06661227, -0.01632868, ..., -0.01999032,
        -0.05139988, -0.0263568 ],
       [-0.01319846, -0.06733431, -0.01605646, ..., -0.0226614 ,
        -0.05537301, -0.02600443],
       [-0.01759106, -0.07094341, -0.01443494, ..., -0.02457913,
        -0.05956192, -0.0231829 ],
       ...,
       [-0.0231029 , -0.05878259, -0.01048967, ..., -0.01945743,
        -0.02615411, -0.02118432],
       [-0.0490171 , -0.05614787, -0.00465348, ..., -0.01065376,
        -0.01797333, -0.02187675],
       [-0.00646111, -0.0914881 , -0.00254872, ..., -0.01505679,
        -0.05040044,  0.04597744]], dtype=float32)

# Fitting

In [4]:
def new_model(embedding_matrix):
    
    inp = Input(shape=(128, ))
    inp2= Input(shape=(1,))

    x = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],weights=[embedding_matrix],trainable=False)(inp)

    x = CuDNNLSTM(150, return_sequences=True,name='lstm_layer',)(x)
    x = CuDNNLSTM(100, return_sequences=False,name='lstm_layer-2',)(x)
    
    y =Dense(10,activation='relu')(inp2)
    x= concatenate([x,y])
    
    x = Dense(108,activation='sigmoid')(x)

    model = Model(inputs=[inp,inp2], outputs=x)

    model.compile(loss='binary_crossentropy',
                      optimizer='adam')


    #print(model.summary())
    
    return model

In [5]:
%%time
model=new_model(embedding_matrix_)
history=model.fit([x_train,sent_train],targets,epochs=3)

Epoch 1/3
27486/27486 [==============================] - 15s 540us/step - loss: 0.1992
Epoch 2/3
27486/27486 [==============================] - 14s 509us/step - loss: 0.1839
Epoch 3/3
27486/27486 [==============================] - 14s 516us/step - loss: 0.1738
CPU times: user 49.5 s, sys: 2.19 s, total: 51.7 s
Wall time: 45.8 s


# 予測

In [6]:
predictions=model.predict([x_test,sent_test])
predictions

array([[3.8938224e-01, 3.7868121e-01, 3.6677265e-01, ..., 5.3787087e-05,
        3.7194259e-05, 3.3745622e-05],
       [3.8938224e-01, 3.7868121e-01, 3.6677265e-01, ..., 5.3787087e-05,
        3.7194259e-05, 3.3745622e-05],
       [3.8938224e-01, 3.7868121e-01, 3.6677265e-01, ..., 5.3787087e-05,
        3.7194259e-05, 3.3745622e-05],
       ...,
       [7.5884783e-01, 7.7701634e-01, 7.8608567e-01, ..., 7.2757000e-05,
        4.6525616e-05, 4.6761677e-05],
       [4.1552097e-01, 4.6107051e-01, 4.7167277e-01, ..., 8.2278282e-05,
        5.2860054e-05, 6.3772437e-05],
       [7.5884783e-01, 7.7701634e-01, 7.8608567e-01, ..., 7.2757000e-05,
        4.6525616e-05, 4.6761677e-05]], dtype=float32)

In [11]:
predictions.shape

(3535, 108)

In [30]:
predictions[4,:]

array([7.58847833e-01, 7.77016342e-01, 7.86085665e-01, 7.64641345e-01,
       7.58248627e-01, 7.20297158e-01, 6.95834398e-01, 6.68446422e-01,
       6.61342382e-01, 6.15113854e-01, 5.65887928e-01, 5.23644984e-01,
       4.97315288e-01, 4.76465940e-01, 4.32322592e-01, 4.03715312e-01,
       3.77745301e-01, 3.44402552e-01, 3.19253832e-01, 2.88284987e-01,
       2.56594956e-01, 2.41797447e-01, 2.20170066e-01, 2.24476099e-01,
       2.07839474e-01, 1.87538370e-01, 1.71712682e-01, 1.40087008e-01,
       1.29910186e-01, 1.17845744e-01, 1.16190627e-01, 8.90396535e-02,
       6.86619729e-02, 6.15128092e-02, 6.00077957e-02, 5.44398390e-02,
       4.95415032e-02, 3.26993428e-02, 2.25041304e-02, 2.04048082e-02,
       1.92981958e-02, 1.96673945e-02, 2.02061571e-02, 1.91855207e-02,
       1.66900810e-02, 1.40940268e-02, 1.57789402e-02, 1.27899833e-02,
       1.44392988e-02, 1.25995865e-02, 1.27851088e-02, 1.32514229e-02,
       1.18172839e-02, 1.08670639e-02, 1.04426015e-02, 1.01303495e-02,
      

In [31]:
test_df.loc[4,"text"]

' 26th February'

In [7]:
def convert_output(sub,predictions):
    preds=[]
    for i,row in enumerate(sub['text']):

        text,target=row.lower(),predictions[i].tolist()
        target=np.round(target).tolist()
        try:
            start,end=target.index(1),target[::-1].index(1)
            text_list=tokenizer.tokenize(text)
            text_list=text_list+((108-len(text_list))*['pad'])
            start_w,end_w=text_list[start],text_list[-end]
            start=text.find(start_w.replace("#",'',1))    ## remove # to match substring
            end=text.find(end_w.replace("#",''),start)
            #pred=' '.join([x for x in text_list[start:-end]])
            pred=text[start:end]
        except:
            pred=text
        
        preds.append(pred)
        
    return preds


In [23]:
test_df = pd.read_csv('/home/tidal/ML_Data/Tweet_Sentiment_Extraction/test.csv')
prediction_text=convert_output(test_df,predictions)
len(prediction_text)
prediction_text

[" http://twitpic.com/67swx - i wish i was calling you but i can't from malta",
 "i'm done.haha. house md marathon ulet",
 " i'm concerned for that family",
 "hey guys it's working no need to worry. i have tooo many followers tho ...",
 ' 26th february',
 'tracy and berwick breaks my achy breaky heart  they split ways in the hallways.',
 "well off 2 bed...cant wait 2 party 4 mother's day in like 14 hours or so",
 'oh yeah the camera clipping problems with void are now completely fixed  yay me for fiddling about',
 "_layne  hmm.. what's ur fav movie?? tv shows??",
 ' salt and vinegar, cheese and onion make your breathe smell lol  ****',
 " nope, i'm on my way home now.",
 'anyone have a super nintendo controller they want to sell me? mine broke.',
 'in study hall with brittneyy!!',
 'is working until 11 tonight  http://plurk.com/p/x2kbv',
 'watched yes man, it was good  http://tinyurl.com/dbrc88',
 " thank u sweetie! can't wait to set sail with u next week",
 '  still need to see that m

In [9]:
sub=pd.read_csv("/home/tidal/ML_Data/Tweet_Sentiment_Extraction/sample_submission.csv")
sub['selected_text']=prediction_text
sub

,textID,selected_text
0,11aa4945ff,http://twitpic.com/67swx - i wish i was calli...
1,fd1db57dc0,i'm done.haha. house md marathon ulet
2,2524332d66,i'm concerned for that family
3,0fb19285b2,hey guys it's working no need to worry. i have...
4,e6c9e5e3ab,26th february
5,311d2b185b,tracy and berwick breaks my achy breaky heart ...
6,95dfefd4e7,well off 2 bed...cant wait 2 party 4 mother's ...
7,739f17cfe1,oh yeah the camera clipping problems with void...
8,c6322a85c2,_layne hmm.. what's ur fav movie?? tv shows??
9,b4401d6b4d,"salt and vinegar, cheese and onion make your ..."


In [22]:
test_df

,textID,text,sentiment
0,11aa4945ff,http://twitpic.com/67swx - i wish i was calli...,positive
1,fd1db57dc0,i'm done.haha. HOUSE MD marathon ulet,positive
2,2524332d66,I'm concerned for that family,positive
3,0fb19285b2,HEY GUYS IT'S WORKING NO NEED TO WORRY. i have...,positive
4,e6c9e5e3ab,26th February,neutral
5,311d2b185b,Tracy and Berwick breaks my achy breaky heart ...,negative
6,95dfefd4e7,Well off 2 bed...cant wait 2 party 4 Mother's ...,positive
7,739f17cfe1,Oh yeah the camera clipping problems with Void...,positive
8,c6322a85c2,_Layne hmm.. what's ur fav movie?? tv shows??,neutral
9,b4401d6b4d,"salt and vinegar, cheese and onion make your ...",negative


# neutralだけそのままにしてみる -②

In [12]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

def jaccard_score(txt_list1, txt_list2):
    jcc_score = 0
    for txt1, txt2 in zip(txt_list1, txt_list2):
        jcc_score += jaccard(txt1,txt2)
        #print(jcc_score)

    jcc_score = jcc_score/len(txt_list1)
    return jcc_score

In [13]:
jaccard_score(["A B C D","Q A Z W"],["A C D H","Q A T G"])

0.4666666666666667

In [14]:
sub.head()

,textID,selected_text
0,11aa4945ff,http://twitpic.com/67swx - i wish i was calli...
1,fd1db57dc0,i'm done.haha. house md marathon ulet
2,2524332d66,i'm concerned for that family
3,0fb19285b2,hey guys it's working no need to worry. i have...
4,e6c9e5e3ab,26th february


In [15]:
test_neutral = test_df[test_df['sentiment'] == 'neutral']
test_neutral.head(10)

,textID,text,sentiment
4,e6c9e5e3ab,26th February,neutral
8,c6322a85c2,_Layne hmm.. what's ur fav movie?? tv shows??,neutral
10,934ac753a2,"Nope, I'm on my way home now.",neutral
12,a8e2c45625,In Study Hall with Brittneyy!!,neutral
13,d761530c7f,is working until 11 tonight http://plurk.com/...,neutral
16,d5c25dbc63,still need to see that movie ! I am all abou...,neutral
17,7a897c9999,http://twitpic.com/4wptj one of my prized mag...,neutral
18,4d7def819d,"Aw, sorry E. :/ I hope it looks up for you (l...",neutral
24,390f169584,im a lady and ladies don't move couches. they...,neutral
25,0de656eac1,**** fiesta at my house in my mouth hit me up...,neutral


In [16]:
test_text_list = test_df.loc[:,'text']
sub_text_list = sub.loc[:,'selected_text']
print(test_text_list[13])
print(sub_text_list[13])
print(jaccard(test_text_list[25],sub_text_list[25]))

is working until 11 tonight  http://plurk.com/p/x2kbv
is working until 11 tonight  http://plurk.com/p/x2kbv
1.0


In [17]:
test_neutral[test_neutral['textID'] == "e6c9e5e3ab"]

,textID,text,sentiment
4,e6c9e5e3ab,26th February,neutral


In [18]:
sub_neutral = pd.DataFrame()
for test_textID in test_neutral['textID']:
    sub_neutral = sub_neutral.append(sub[sub['textID'] == test_textID])

print(sub_neutral)


          textID                                      selected_text
4     e6c9e5e3ab                                      26th february
8     c6322a85c2     _layne  hmm.. what's ur fav movie?? tv shows??
10    934ac753a2                      nope, i'm on my way home now.
12    a8e2c45625                     in study hall with brittneyy!!
13    d761530c7f  is working until 11 tonight  http://plurk.com/...
...          ...                                                ...
3523  eabaf375f4  i must have viewed that picture of me and  fro...
3526  885b8f47f7                                yes sir i sure did.
3529  44daf50a02               paper writing til my heart's content
3532  08f6036add  invasion of the old ladies has just ended, sti...
3534  aa1a163174                                            will do

[1430 rows x 2 columns]


In [19]:
print(type(sub_neutral))
print(type(test_neutral))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [20]:
test_text_list_ntl = test_neutral.loc[:,'text']
sub_text_list_ntl = sub_neutral.loc[:,'selected_text']

print(jaccard_score(test_text_list_ntl, sub_text_list_ntl))
    

1.0


In [21]:
test_text_list = test_df.loc[:,'text']
sub_text_list = sub.loc[:,'selected_text']

print(jaccard_score(test_text_list, sub_text_list))

1.0


# ここまで -②

In [21]:
sub.to_csv('/home/tidal/ML_Data/Tweet_Sentiment_Extraction/submit/submission.csv',index=False)
sub.head()

,textID,selected_text
0,11aa4945ff,http://twitpic.com/67swx - i wish i was calli...
1,fd1db57dc0,i'm done.haha. house md marathon ulet
2,2524332d66,i'm concerned for that family
3,0fb19285b2,hey guys it's working no need to worry. i have...
4,e6c9e5e3ab,26th february
